### ****** 필독 !! 실습과제 주의사항 ******

- 각자의 "수강생 번호" 확인 (아이캠퍼스 공지) 
- 제출하는 실습과제 파일에 "수강생 번호"를 기준으로 작성 (이름, 학번 등 작성X)
- 각 실습과제에 대한 구체적인 파일 이름은 매번 개별 안내 
  (수강생 번호 123번 학생 과제파일 예시 : 'ST123_HW2_1' )
  
- 과제 파일이름 양식 지키지 않을 시 감점!

# [실습 과제 2]

## 1. 센서 데이터 1개 (가속도) 및 WT level 4으로 설정하여 다른 형태의 특징데이터 추출 
#### >>>>>> TimeDomain 10개 + FrequencyDomain 40개 = 총 50개 특징 추출 50행 360열
#### >>>>>> 추출된 데이터를 파일로 저장 후 제출
#### >>>>>> 데이터 파일 이름 : ST(수강생 번호)_HW2_1  (예시 : 'ST00_HW2_1'  //  'ST0_HW2_1')
#### >>>>>> 데이터 파일 이름 중 'ST' , 'HW' 등 영어는 모두 대문자

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt

In [ ]:
NoOfData    = 180  # 정상/고장 데이터 각 180개씩 
NoOfSensor  = 1    # 가속도(Acceleration), 전압(Voltage), 전류(Current)
NoOfFeature = 10   # 특징 개수:10개 (순서: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Shape factor, Impulse factor)

NoOfData, NoOfSensor, NoOfFeature

In [ ]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [ ]:
TimeFeature_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Abnormal = np.zeros((NoOfSensor*NoOfFeature , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',' , header=None) # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j+1])/rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j+1])/np.mean(np.abs(temp_data1.iloc[:,j+1]))
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j+1])/np.mean(np.abs(temp_data1.iloc[:,j+1]))
        
        # Abnormal Time Domain Feature
        TimeFeature_Abnormal[10*j+0, i] = np.max(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+1, i] = np.min(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+2, i] = np.mean(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+3, i] = rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+4, i] = np.var(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+5, i] = sp.skew(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+7, i] = np.max(temp_data2.iloc[:,j+1])/rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+8, i] = rms(temp_data2.iloc[:,j+1])/np.mean(np.abs(temp_data2.iloc[:,j+1]))
        TimeFeature_Abnormal[10*j+9, i] = np.max(temp_data2.iloc[:,j+1])/np.mean(np.abs(temp_data2.iloc[:,j+1]))
        
print(TimeFeature_Normal.shape)
print(TimeFeature_Abnormal.shape)

In [ ]:
TimeFeature = np.concatenate([TimeFeature_Normal, TimeFeature_Abnormal] , axis=1)
TimeFeature.shape

In [ ]:
print(TimeFeature)

In [ ]:
MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 4                            # Wavelet 분해 레벨 지정
select  = 4                            # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [ ]:
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))
FreqFeature_Abnormal = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1 , sep=',', header=None).iloc[:,1:]) # 임시 Normal 데이터
    temp_data2 = np.array(pd.read_csv(temp_path2 , sep=',', header=None).iloc[:,1:]) # 임시 Abnormal 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_data2, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
            
            # Normal Frequency Domain Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(np.abs(coef1[:,j]))
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(np.abs(coef1[:,j]))
            
            # Abnormal Frequency Domain Feature
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef2[:,j])/np.mean(np.abs(coef2[:,j]))
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef2[:,j])/np.mean(np.abs(coef2[:,j]))

print(FreqFeature_Normal.shape)
print(FreqFeature_Abnormal.shape)

In [ ]:
FreqFeature = np.concatenate([FreqFeature_Normal, FreqFeature_Abnormal] , axis=1)
FreqFeature.shape

In [ ]:
Features = np.concatenate([TimeFeature,FreqFeature] , axis=0)

print("Feature Data Size :", Features.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 특징값으로 구성됨"%(Features.shape[1], Features.shape[1]/2, Features.shape[0]))

In [ ]:
FeatureData = pd.DataFrame(Features)
FeatureData

In [ ]:
path = './ProcessedData1/FeatureData'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)

In [ ]:
path = './ProcessedData1/ST41_HW2_1'
FeatureData.to_csv(path , sep=',' , header=None , index=None)

## 2. 위의 특징데이터에서 P-value 오름차순 상/하위 각각 5개를 선택한 2가지 특징데이터 추출
#### >>>>>> 추출된 2개 데이터를 각각의 파일로 저장 후 제출
#### >>>>>> 데이터 파일 이름 : ST(수강생 번호)_HW2_2 ,  ST(수강생 번호)_HW2_3 (예시 : 'ST00_HW2_2(or 3)'  //  'ST0_HW2_2(or 3)')
#### >>>>>> 데이터  파일 이름 중 'ST' , 'HW' 등 영어는 모두 대문자

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sb
import scipy.stats       as sp
import matplotlib.pyplot as plt

In [ ]:
FeatureData = pd.read_csv('./ProcessedData1/ST41_HW2_1', sep=',', header=None)
FeatureData.shape

In [ ]:
NoOfData = int(FeatureData.shape[1]/2)
Normal_FeatureData   = FeatureData.iloc[:,:NoOfData]
Abnormal_FeatureData = FeatureData.iloc[:,NoOfData:]

print(Normal_FeatureData.shape)
print(Abnormal_FeatureData.shape)

In [ ]:
NoOfFeature = FeatureData.shape[0] # 추출된 Feature 갯수

P_value = np.zeros((NoOfFeature , 2))

# 특징값 각각 T-검정 수행
for i in np.arange(NoOfFeature):
    
    T_test       = np.array(sp.ttest_ind(Normal_FeatureData.iloc[i,:] , Abnormal_FeatureData.iloc[i,:]))
    P_value[i,0] = i          # Feature Index
    P_value[i,1] = T_test[1]  # P값 (P-value)
    
P_value      = pd.DataFrame(P_value)
P_value

In [ ]:
P_value_Rank = P_value.sort_values([1],ascending=False)  # P-value 기준 오름차순 정렬

P_value_Rank

In [ ]:
path1 = './ProcessedData1/P_value'        # path = '파일 경로/저장할 파일 이름'
path2 = './ProcessedData1/P_value_Rank'   # path = '파일 경로/저장할 파일 이름'
P_value.to_csv(path1, sep=',', header=None, index=None)
P_value_Rank.to_csv(path2, sep=',', header=None, index=None)

In [ ]:
Rank = 5

Normal   = np.zeros((Rank,NoOfData))
Abnormal = np.zeros((Rank,NoOfData))

for i in range(Rank):
    
    index         = int(P_value_Rank.iloc[i,0])
    Normal[i,:]   = Normal_FeatureData.iloc[index,:].values
    Abnormal[i,:] = Abnormal_FeatureData.iloc[index,:].values

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Abnormal] , axis=1))

print("Selected Feature Data Size :", FeatureSelected.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 최종 선택된 특징값으로 구성됨"
      %(FeatureSelected.shape[1], FeatureSelected.shape[1]/2, FeatureSelected.shape[0]))

In [ ]:
path = './ProcessedData1//ST41_HW2_2'   # path = '파일 경로//저장할 파일 이름'
FeatureSelected.to_csv(path, sep=',', header=None, index=None)

In [ ]:
print(FeatureSelected)

## 3. 위의 2가지 특징데이터를 이용하여 t-SNE 수행 및 그래프 그리기
#### >>>>>> P-value 기준 상위 5개 특징데이터 및 하위 5개 특징데이터 각각 t-SNE 수행 및 그래프 출력 (정상 : 파랑 // 고장 : 빨강)
#### >>>>>> 출력된 그래프를 각각 캡쳐/저장하고, 이미지 파일(.jpg) 2개 제출 t-sne파라미터 변경전
#### >>>>>> 이미지 파일 이름 : ST(수강생 번호)_HW2_상위특징.jpg , ST(수강생 번호)_HW2_하위특징.jpg
#### >>>>>> 이미지 파일 이름 중 'ST' , 'HW' 등 영어는 모두 대문자

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import time

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold      import TSNE

In [ ]:
FeatureData = pd.read_csv('./ProcessedData1/FeatureData', header=None)
FeatureData.shape

In [ ]:
P_value_Rank = pd.read_csv('./ProcessedData1/P_value_Rank' , header=None)
P_value_Rank

In [ ]:
# StartRank 부터 Number 만큼의 Feature
StartRank = 1
Number    = 5

SelectedFeatues = np.zeros((Number,FeatureData.shape[1]))

s = 0

for i in range(StartRank, StartRank+Number):
    
    index                = int(P_value_Rank.iloc[i-1,0])
    SelectedFeatues[s,:] = FeatureData.iloc[index,:].values
    s += 1

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(SelectedFeatues)
FeatureSelected.shape

In [ ]:
FeatureSelected = np.transpose(FeatureSelected)
FeatureSelected.shape

In [ ]:
time_start   = time.time()
tsne         = TSNE(n_components=2, verbose=1, perplexity=20, n_iter=500, random_state=1) #perplexity 주변의 몇개의 점까지 할지
tsne_results = tsne.fit_transform(FeatureSelected)

print( '\n\n t-SNE done! Time elapsed: {} seconds'.format(time.time() - time_start )) #kl다이버젼스 에러라고 생각 낮추는 방향으로

In [ ]:
pd.DataFrame(tsne_results)

In [ ]:
NoOfData = int(FeatureSelected.shape[0]/2)

plt.figure(figsize=(10,5))

plt.scatter(tsne_results[:NoOfData,0], tsne_results[:NoOfData,1], marker='o', label='Normal')
plt.scatter(tsne_results[NoOfData:,0], tsne_results[NoOfData:,1], marker='o', label='Abnormal', c='r')
plt.title('t-SNE Result', fontsize=15)
plt.grid(alpha=0.5)
plt.legend(fontsize=12)
plt.xlabel('t-SNE_1')
plt.ylabel('t-SNE_2')

plt.show()

## ***** 데이터 파일 및 이미지 파일 함께 zip파일로 압축하여 제출
### >>> 압축파일 이름 ST(수강생 번호)_HW2 (예시 : 'ST00_HW2'  //  'ST0_HW2')